In [13]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# for local installation
# sys.path.append("../") # go to parent dir if running notebook directly in sub-folder

# for Google Colab
!git clone http://github.com/tjbarnum13/Fourier-DVR-1D.git
sys.path.insert(0,'/content/Fourier-DVR-1D')

from fourier_DVR_1D import Domain_Fourier_DVR_1D

fatal: destination path 'Fourier-DVR-1D' already exists and is not an empty directory.


In [14]:
# settings
x_min = -5.0
x_max = 5.0
n_DVR = 300
n_g = 1001
scale_wfn=3

In [15]:
domain = Domain_Fourier_DVR_1D(x_min, x_max, n_DVR)

In [20]:
def run(yrange, n_plot, m, well_length, well_depth):

    # define the potential
    well_lowerlimit=-well_length/2
    well_upperlimit=well_length/2

    def V(x):
      y = np.piecewise(
          x,
          [x < well_lowerlimit, (x >= well_lowerlimit) & (x <= well_upperlimit), x > well_upperlimit],
          [well_depth, 0, well_depth],
          )
      return y
    
    # solve
    E, E_four = domain.solve(m, V)

    # evaluate eigenstates on grid
    x = np.linspace(x_min, x_max, n_g)
    psi_x = domain.grid(x, E_four[:,:n_plot])

    # plot eigenstates
    plt.figure(figsize=(10, 6))
    for i in range(n_plot):
        plt.plot([x[0], x[-1]], [E[i], E[i]], '-', color='gray', lw=1.0)
    plt.plot(x, V(x), 'k-', lw=2)
    for i in range(n_plot):
        if E[i] < yrange[1]:
          plt.plot(x, scale_wfn * psi_x[i] + E[i], lw=1.0)
          plt.text(well_upperlimit+0.1,E[i]+0.1,'n=%i, E=%0.1f' % (i, E[i]))
    plt.xlim(x_min, x_max)
    plt.ylim(yrange)
    plt.xlabel('position')
    plt.ylabel('energy')

    plt.tight_layout()

    plt.show()

In [21]:
cu = False # turn off continous update for smoother operation

# set up all the sliders to change variables
yrange=widgets.IntRangeSlider(value=[-3, 20],min=-5,max=500,step=1,description='Y-Range:',continuous_update=cu)
n_plot=widgets.IntSlider(min=1, max=20, step=1, value=3, description='# States:',continuous_update=cu)
m=widgets.FloatSlider(min=0.1, max=2, step=0.1, value=1.0, description='Mass:',continuous_update=cu)
well_length=widgets.FloatSlider(min=0.1, max=x_max, step=0.1, value=2, description='Well Length:',continuous_update=cu)
well_depth=widgets.FloatLogSlider(value=1e5, base=10, min=0, max=5, step=0.01, description='Well Depth:', continuous_update=cu)

# define the output and arrange sliders and plot
out = widgets.interactive_output(run,{'yrange':yrange,'n_plot':n_plot,'m':m,'well_length':well_length,'well_depth':well_depth})
controls = widgets.VBox([yrange, n_plot, m, well_length, well_depth])
display(widgets.HBox([controls, out]))